In [3]:
import os
import re

In [4]:
!pip install pandas

In [5]:
import pandas as pd

In [83]:
# !pip install psycopg2
# import psycopg2

In [84]:
# %load_ext sql

In [6]:
HOST = "localhost"
DB = "tweetsDB"
USER = "postgres"
PASSWD = "passwd"

In [86]:
# conn = psycopg2.connect(dbname=DB, user=USER, password=PASSWD, host=HOST)

In [87]:
# cur = conn.cursor()

In [88]:
# cur.execute(query)

In [89]:
# cur.fetchall()

In [90]:
# res = cur.execute("SELECT * FROM hashtags LIMIT 100")

In [91]:
# print(res)

In [7]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, String, Integer, MetaData
from sqlalchemy.orm import declarative_base, sessionmaker

In [8]:
posgres_conn_str = f"postgresql://{USER}:{PASSWD}@{HOST}/{DB}"

In [9]:
engine = create_engine(posgres_conn_str)

TODO:

    1. vytvorit si novy stlpec pre sentiment(neg, neu, pos, compound)
    2. joinut tweety s tweethashtagmi s hashtagmi 
    3. vyfiltrovat podla danych hashtagov, ktore chcem 
    4. vypocitat sentiment a zapisat do DB

Adding the sentiment columns to the tweets table

In [266]:
add_columns_query = """

    BEGIN TRAN 

    ALTER TABLE tweets
    ADD COLUMN neg numeric(6, 4),
    ADD COLUMN neu numeric(6, 4),
    ADD COLUMN pos numeric(6, 4),
    ADD COLUMN compound numeric(6, 4)

    COMMIT TRAN 

"""

In [8]:
try:
    engine.execute(add_columns_query)
    engine.commit()
except:
    print("Columns probably already exist")

Columns probably already exist


In [10]:
HASHTAG_FILTER = """'%%DeepstateVirus%%', '%%DeepStateVaccine%%','%%DeepStateFauci%%', '%%QAnon%%', '%%Agenda21%%', '%%CCPVirus%%', '%%ClimateChangeHoax%%',
'%%GlobalWarmingHoax%%', '%%ChinaLiedPeopleDied%%', '%%SorosVirus%%', '%%5GCoronavirus%%', '%%MAGA%%',
'%%WWG1WGA%%', '%%Chemtrails%%', '%%flatEarth%%', '%%MoonLandingHoax%%', '%%moonhoax%%', '%%illuminati%%',
'%%pizzaGateIsReal%%', '%%PedoGateIsReal%%', '%%911truth%%', '%%911insidejob%%', '%%reptilians%%'"""

In [32]:
# hashtag : conspiracy_theory
CONSPIRACY_THEORIES = {
 
        "deepstatevirus"                :   "Deep State",  
        "deepstatevaccine"              :   "Deep State",
        "deepstatefauci"                :   "Deep State",
        "qanon"                         :   "Qanon",
        "maga"                          :   "Qanon",
        "wwg1wga"                       :   "Qanon",
        "agenda21"                      :   "New world order",
        "ccpvirus"                      :   "The virus escaped from a Chinese lab", 
        "chinaliedpeopledied"           :   "The virus escaped from a Chinese lab",
        "climatechangehoax"             :   "GLobal Warming is HOAX",
        "globalwarminghoax"             :   "GLobal Warming is HOAX",
        "sorosvirus"                    :   "COVID19 and microchipping", 
        "billgates"                     :   "COVID19 and microchipping",
        "5gcoronavirus"                 :   "COVID19 is preaded by 5G",
        "moonlandinghoax"               :   "Moon landing is fake", 
        "moonhoax"                      :   "Moon landing is fake",
        "911truth"                      :   "9/11 was inside job",
        "911insidejob"                  :   "9/11 was inside job",
        "pizzagateisreal"               :   "Pizzagate conspiracy theory",
        "pedogateisreal"                :   "Pizzagate conspiracy theory",   
        "chemtrails"                    :   "Chemtrails",
        "flatearth"                     :   "FlatEarth",
        "illuminati"                    :   "Illuminati",
        "reptilians"                    :   "Reptilian conspiracy theory"
}

In [12]:
CONSPIRACY_THEORIES["DeepStateVaccine"]

'Deep State'

We need to match all the mutations of the hashtags:

In [272]:
filter_query = f"""
    SELECT * 
    FROM hashtags
    WHERE lower(value) LIKE ANY (array[{HASHTAG_FILTER.lower()}])
"""

In [273]:
filter_query

"\n    SELECT * \n    FROM hashtags\n    WHERE lower(value) LIKE ANY (array['%%deepstatevirus%%', '%%deepstatevaccine%%','%%deepstatefauci%%', '%%qanon%%', '%%agenda21%%', '%%ccpvirus%%', '%%climatechangehoax%%',\n'%%globalwarminghoax%%', '%%chinaliedpeopledied%%', '%%sorosvirus%%', '%%5gcoronavirus%%', '%%maga%%',\n'%%wwg1wga%%', '%%chemtrails%%', '%%flatearth%%', '%%moonlandinghoax%%', '%%moonhoax%%', '%%illuminati%%',\n'%%pizzagateisreal%%', '%%pedogateisreal%%', '%%911truth%%', '%%911insidejob%%', '%%reptilians%%'])\n"

In [274]:
qanon_query = f"""
    SELECT * 
    FROM hashtags
    WHERE lower(value) LIKE ANY (array['qanon'])
"""

In [275]:
df_qanon = pd.read_sql(qanon_query, engine)

In [276]:
df_filtered = pd.read_sql(filter_query, engine)

In [277]:
df_filtered

,id,value
0,3073063,CCPVirus
1,1137275,CCPvirus
2,15552438,5GCoronavirus
3,23142030,bostonathletemagazine
4,23146777,5GChemtrails
...,...,...
2013,23111551,impeachimagain
2014,23111710,MAGATruthBomb
2015,363800,magats
2016,10905016,ccpvirus


In [278]:
df_qanon

,id,value
0,21259243,QAnON
1,15690756,qANON
2,15177935,qAnon
3,16799781,qanOn
4,18387797,QanON
5,27195,QANON
6,22837040,QanoN
7,25091,qanon
8,22849188,QanOn
9,15168832,QANOn


We can see that hoaxer dont know grammar

In [279]:
emoji_pat = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                        "]+", flags = re.UNICODE)


def parse_line(line):
    res = re.sub("(\#|\@)[\w]+", "", line)
    res = emoji_pat.sub(r'',res)
    return res.lstrip().rstrip()

In [315]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


def compute_sentiment_and_write(line):
    # parse the line and compute the sentiment
    id = line['id']
    sentiment = sid.polarity_scores(parse_line(line['content']))
    # print(parse_line(line['content']))
    # print(sid.polarity_scores(parse_line(line['content'])))
    try:
        engine.execute(f"""
            BEGIN TRAN 

            UPDATE tweets
            SET neg = {sentiment['neg']},
                neu = {sentiment['neu']},
                pos = {sentiment['pos']},
                compound = {sentiment['compound']}
            WHERE id = '{id}';

            COMMIT TRAN 
        """)
        
    except:
        print("Something went wrong during write")




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kdeni\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
lower_hastag_filter = HASHTAG_FILTER.lower()

In [13]:
lower_hastag_filter

"'%%deepstatevirus%%', '%%deepstatevaccine%%','%%deepstatefauci%%', '%%qanon%%', '%%agenda21%%', '%%ccpvirus%%', '%%climatechangehoax%%',\n'%%globalwarminghoax%%', '%%chinaliedpeopledied%%', '%%sorosvirus%%', '%%5gcoronavirus%%', '%%maga%%',\n'%%wwg1wga%%', '%%chemtrails%%', '%%flatearth%%', '%%moonlandinghoax%%', '%%moonhoax%%', '%%illuminati%%',\n'%%pizzagateisreal%%', '%%pedogateisreal%%', '%%911truth%%', '%%911insidejob%%', '%%reptilians%%'"

In [56]:
query = f"""
                SELECT tweets.id as tweetID, tweets.content, h.id as hashtagID, h.value as hashtagVal
                FROM tweets
                JOIN tweet_hashtags as th ON th.tweet_id = tweets.id
                JOIN hashtags as h ON h.id = th.hashtag_id
                WHERE lower(h.value) ILIKE ANY (array[{lower_hastag_filter}])
        
"""

In [284]:
query_hash = f"""
        SELECT value
        FROM hashtags as h
        WHERE lower(h.value) ILIKE ANY (array[{lower_hastag_filter}])
"""

In [285]:
query_hash

"\n        SELECT value\n        FROM hashtags as h\n        WHERE lower(h.value) ILIKE ANY (array['%%deepstatevirus%%', '%%deepstatevaccine%%','%%deepstatefauci%%', '%%qanon%%', '%%agenda21%%', '%%ccpvirus%%', '%%climatechangehoax%%',\n'%%globalwarminghoax%%', '%%chinaliedpeopledied%%', '%%sorosvirus%%', '%%5gcoronavirus%%', '%%maga%%',\n'%%wwg1wga%%', '%%chemtrails%%', '%%flatearth%%', '%%moonlandinghoax%%', '%%moonhoax%%', '%%illuminati%%',\n'%%pizzagateisreal%%', '%%pedogateisreal%%', '%%911truth%%', '%%911insidejob%%', '%%reptilians%%'])\n"

In [286]:
df_some = pd.read_sql(query_hash, engine)

In [287]:
df_some

,value
0,CCPVirus
1,5GCoronavirus
2,CCPvirus
3,bostonathletemagazine
4,5GChemtrails
...,...
2013,MAGATruthBomb
2014,magats
2015,magazine
2016,WeddingMagazine


In [57]:
df_filtered_tweets = pd.read_sql(query, engine)

In [16]:
df_filtered_tweets

,tweetid,content,hashtagid,hashtagval
0,1263951475576692736,@ZcohenCNN @RepAdamSchiff #obamagate is just t...,17457016,obamagate
1,1263951734197547008,@catturd2 Welcome to the [D] party! 🤮\r\n\r\n#...,20506,QAnon
2,1263951734197547008,@catturd2 Welcome to the [D] party! 🤮\r\n\r\n#...,25089,WWG1WGA
3,1263951707970719747,Conspiracy Theorist\r\n\r\nA term CREATED by...,20506,QAnon
4,1263951707240837120,I took the liberty of deconstructing this week...,25089,WWG1WGA
...,...,...,...,...
3656137,1263951761347354624,Avivim School Bus Terror Attack 50 Years Later...,20506,QAnon
3656138,1263951761347354624,Avivim School Bus Terror Attack 50 Years Later...,14415,MAGA
3656139,1263857555060686848,Jessica Long  Couldnt do it alone #IamSecon...,20506,QAnon
3656140,1263904736580952065,White House coronavirus press conference  05/...,20506,QAnon


In [17]:
df_filtered_tweets['hashtagval'] = df_filtered_tweets['hashtagval'].apply(lambda x: x.lower()) 

Remove duplicates

In [21]:
df_unique_filtered_tweets = df_filtered_tweets.drop_duplicates(subset=['tweetid'])

In [22]:
df_unique_filtered_tweets

,tweetid,content,hashtagid,hashtagval
0,1263951475576692736,@ZcohenCNN @RepAdamSchiff #obamagate is just t...,17457016,obamagate
1,1263951734197547008,@catturd2 Welcome to the [D] party! 🤮\r\n\r\n#...,20506,qanon
3,1263951707970719747,Conspiracy Theorist\r\n\r\nA term CREATED by...,20506,qanon
4,1263951707240837120,I took the liberty of deconstructing this week...,25089,wwg1wga
6,1263951688433623042,#wwg1wga #qanon | (ಠ_ಠ)+n+@dananessel \r\n\r\n...,511508,wwg1wga
...,...,...,...,...
3656136,1263845894178385921,The CCP must pay dearly if it crushes Hong Kon...,20506,qanon
3656137,1263951761347354624,Avivim School Bus Terror Attack 50 Years Later...,20506,qanon
3656139,1263857555060686848,Jessica Long  Couldnt do it alone #IamSecon...,20506,qanon
3656140,1263904736580952065,White House coronavirus press conference  05/...,20506,qanon


In [23]:
df_filtered_tweets

,tweetid,content,hashtagid,hashtagval
0,1263951475576692736,@ZcohenCNN @RepAdamSchiff #obamagate is just t...,17457016,obamagate
1,1263951734197547008,@catturd2 Welcome to the [D] party! 🤮\r\n\r\n#...,20506,qanon
2,1263951734197547008,@catturd2 Welcome to the [D] party! 🤮\r\n\r\n#...,25089,wwg1wga
3,1263951707970719747,Conspiracy Theorist\r\n\r\nA term CREATED by...,20506,qanon
4,1263951707240837120,I took the liberty of deconstructing this week...,25089,wwg1wga
...,...,...,...,...
3656137,1263951761347354624,Avivim School Bus Terror Attack 50 Years Later...,20506,qanon
3656138,1263951761347354624,Avivim School Bus Terror Attack 50 Years Later...,14415,maga
3656139,1263857555060686848,Jessica Long  Couldnt do it alone #IamSecon...,20506,qanon
3656140,1263904736580952065,White House coronavirus press conference  05/...,20506,qanon


In [244]:
'magazine' in df_filtered_tweets.value.values

True

Computing the sentiment and writing it to the db

In [255]:
id_count = len(df_filtered_tweets)

In [316]:
df_unique_filtered_tweets.apply(lambda x: compute_sentiment_and_write(x), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
3656131    None
3656133    None
3656134    None
3656135    None
3656140    None
Length: 2152424, dtype: object

In [246]:
# for (id, sentiment) in new_col:

#     engine.execute(f"""
#         UPDATE tweets
#         SET neg = {sentiment['neg']},
#             neu = {sentiment['neu']},
#             pos = {sentiment['pos']},
#             compound = {sentiment['compound']}
#         WHERE id = '{id}';
#     """)


Creating the consp_theory tables as well the link between hashtags and consptheory

In [24]:
consp_theory_link = df_filtered_tweets.loc[ : , ["hashtagid", "hashtagval", "tweetid"]]

In [25]:
consp_theory_link

,hashtagid,hashtagval,tweetid
0,17457016,obamagate,1263951475576692736
1,20506,qanon,1263951734197547008
2,25089,wwg1wga,1263951734197547008
3,20506,qanon,1263951707970719747
4,25089,wwg1wga,1263951707240837120
...,...,...,...
3656137,20506,qanon,1263951761347354624
3656138,14415,maga,1263951761347354624
3656139,20506,qanon,1263857555060686848
3656140,20506,qanon,1263904736580952065


In [26]:
consp_theory_link = consp_theory_link.drop_duplicates()

In [27]:
consp_theory_link

,hashtagid,hashtagval,tweetid
0,17457016,obamagate,1263951475576692736
1,20506,qanon,1263951734197547008
2,25089,wwg1wga,1263951734197547008
3,20506,qanon,1263951707970719747
4,25089,wwg1wga,1263951707240837120
...,...,...,...
3656137,20506,qanon,1263951761347354624
3656138,14415,maga,1263951761347354624
3656139,20506,qanon,1263857555060686848
3656140,20506,qanon,1263904736580952065


In [28]:
consp_theory_link[consp_theory_link['tweetid'] == str(1256801279139069952)]

,hashtagid,hashtagval,tweetid
116857,20506,qanon,1256801279139069952
116859,15171991,chemtrails,1256801279139069952
116864,25089,wwg1wga,1256801279139069952
116865,259500,wwg1wgaworldwide,1256801279139069952


In [29]:
consp_theory_link = consp_theory_link.reset_index(drop=True)

In [30]:
consp_theory_link

,hashtagid,hashtagval,tweetid
0,17457016,obamagate,1263951475576692736
1,20506,qanon,1263951734197547008
2,25089,wwg1wga,1263951734197547008
3,20506,qanon,1263951707970719747
4,25089,wwg1wga,1263951707240837120
...,...,...,...
3656137,20506,qanon,1263951761347354624
3656138,14415,maga,1263951761347354624
3656139,20506,qanon,1263857555060686848
3656140,20506,qanon,1263904736580952065


In [33]:
CONSPIRACY_THEORIES.get(consp_theory_link['hashtagval'][2015])

'Qanon'

In [34]:
CONSPIRACY_THEORIES.get("some")

In [35]:
consp_theory_link['hashtagval'][2015]

'wwg1wga'

Get the name of the consp. theory.

In [208]:
consp_theory_link

,hashtagid,hashtagval,tweetid
0,20506,qanon,1263951734197547008
1,25089,wwg1wga,1263951734197547008
2,20506,qanon,1263951707970719747
3,25089,wwg1wga,1263951707240837120
4,77112,qanon,1263951707240837120
...,...,...,...
3656137,20506,qanon,1263894906822709252
3656138,20506,qanon,1263951677348024322
3656139,25089,wwg1wga,1263951677348024322
3656140,14415,maga,1263951990620590081


In [36]:
consp_theory_link['theory'] = consp_theory_link.apply(lambda x: CONSPIRACY_THEORIES.get(x["hashtagval"]), axis=1)

In [211]:
for index, row  in consp_theory_link.iterrows():
    print(row['hashtagid'], row['hashtagval'])

20506 qanon
25089 wwg1wga
20506 qanon
25089 wwg1wga
77112 qanon
511508 wwg1wga
25091 qanon
20506 qanon
77112 qanon
25089 wwg1wga
77112 qanon
77112 qanon
20506 qanon
77112 qanon
25089 wwg1wga
14415 maga
77112 qanon
20506 qanon
15291661 qanongermany
21776040 qanondach
77112 qanon
25089 wwg1wga
259500 wwg1wgaworldwide
20506 qanon
25089 wwg1wga
20506 qanon
14415 maga
20506 qanon
158957 maga2020
20506 qanon
20506 qanon
20506 qanon
20506 qanon
14415 maga
77112 qanon
20506 qanon
17457016 obamagate
25089 wwg1wga
20506 qanon
20506 qanon
14415 maga
25089 wwg1wga
20506 qanon
25089 wwg1wga
20506 qanon
25089 wwg1wga
20506 qanon
25089 wwg1wga
20506 qanon
488582 obamagate
25089 wwg1wga
20506 qanon
77112 qanon
25089 wwg1wga
20506 qanon
25091 qanon
259500 wwg1wgaworldwide
20506 qanon
20506 qanon
20506 qanon
25089 wwg1wga
259500 wwg1wgaworldwide
20506 qanon
20506 qanon
14415 maga
20506 qanon
20506 qanon
158957 maga2020
6484683 maga2020
20506 qanon
20506 qanon
20506 qanon
158957 maga2020
20506 qanon
2508

KeyboardInterrupt: 

In [37]:
consp_theory_link

,hashtagid,hashtagval,tweetid,theory
0,17457016,obamagate,1263951475576692736,None
1,20506,qanon,1263951734197547008,Qanon
2,25089,wwg1wga,1263951734197547008,Qanon
3,20506,qanon,1263951707970719747,Qanon
4,25089,wwg1wga,1263951707240837120,Qanon
...,...,...,...,...
3656137,20506,qanon,1263951761347354624,Qanon
3656138,14415,maga,1263951761347354624,Qanon
3656139,20506,qanon,1263857555060686848,Qanon
3656140,20506,qanon,1263904736580952065,Qanon


In [38]:
consp_values = list(set(CONSPIRACY_THEORIES.values()))

In [39]:
id_arr = [x + 1 for x in range(0, len(consp_values))]

In [40]:
consp_theory_tab = pd.DataFrame(list(zip(id_arr, consp_values)), columns=["id", "theory_name"])

In [41]:
consp_theory_tab

,id,theory_name
0,1,Qanon
1,2,Deep State
2,3,COVID19 and microchipping
3,4,9/11 was inside job
4,5,Illuminati
5,6,Pizzagate conspiracy theory
6,7,COVID19 is preaded by 5G
7,8,Reptilian conspiracy theory
8,9,The virus escaped from a Chinese lab
9,10,Chemtrails


In [304]:
try:
    engine.execute("""
        BEGIN TRAN 

        CREATE TABLE IF NOT EXISTS consp_theories (
        id INT GENERATED ALWAYS AS IDENTITY,
        theory_name varchar(255),
        PRIMARY KEY(id)
    );   

        COMMIT TRAN 
    """)
    # consp_theory_tab.to_sql('consp_theories', engine, if_exists='replace', index=False)

except:
    print("Something went wrong")

Inserting the consp_theories

In [305]:
try:
    for _, row in consp_theory_tab.iterrows():
        # print(row['theory_name'])
        engine.execute(f"""
            BEGIN TRAN 
            INSERT INTO consp_theories(theory_name)
            VALUES ('{row['theory_name']}');
            COMMIT TRAN   
        """)
        # consp_theory_tab.to_sql('consp_theories', engine, if_exists='replace', index=False)
except:
    print("Somthing went wrong")

In [42]:
df_theories = pd.read_sql(
    """
    SELECT id, theory_name
    FROM 
    consp_theories
    """
    , engine)

In [43]:
df_theories['theory_name']

0                      9/11 was inside job
1     The virus escaped from a Chinese lab
2              Reptilian conspiracy theory
3                               Chemtrails
4                   GLobal Warming is HOAX
5              Pizzagate conspiracy theory
6                          New world order
7                                    Qanon
8                               Deep State
9                     Moon landing is fake
10                              Illuminati
11                COVID19 is preaded by 5G
12               COVID19 and microchipping
13                               FlatEarth
Name: theory_name, dtype: object

In [293]:
df_theories

,id,theory_name
0,1,9/11 was inside job
1,2,The virus escaped from a Chinese lab
2,3,Reptilian conspiracy theory
3,4,Chemtrails
4,5,GLobal Warming is HOAX
5,6,Pizzagate conspiracy theory
6,7,New world order
7,8,Qanon
8,9,Deep State
9,10,Moon landing is fake


In [44]:
mapping_dict = dict(zip(df_theories['theory_name'], df_theories['id']))

In [45]:
mapping_dict

{'9/11 was inside job': 1,
 'The virus escaped from a Chinese lab': 2,
 'Reptilian conspiracy theory': 3,
 'Chemtrails': 4,
 'GLobal Warming is HOAX': 5,
 'Pizzagate conspiracy theory': 6,
 'New world order': 7,
 'Qanon': 8,
 'Deep State': 9,
 'Moon landing is fake': 10,
 'Illuminati': 11,
 'COVID19 is preaded by 5G': 12,
 'COVID19 and microchipping': 13,
 'FlatEarth': 14}

In [46]:
consp_theory_link['theory_id'] = consp_theory_link.apply(lambda x: mapping_dict.get(x['theory']), axis=1)

In [47]:
consp_theory_link

,hashtagid,hashtagval,tweetid,theory,theory_id
0,17457016,obamagate,1263951475576692736,None,NaN
1,20506,qanon,1263951734197547008,Qanon,8.0
2,25089,wwg1wga,1263951734197547008,Qanon,8.0
3,20506,qanon,1263951707970719747,Qanon,8.0
4,25089,wwg1wga,1263951707240837120,Qanon,8.0
...,...,...,...,...,...
3656137,20506,qanon,1263951761347354624,Qanon,8.0
3656138,14415,maga,1263951761347354624,Qanon,8.0
3656139,20506,qanon,1263857555060686848,Qanon,8.0
3656140,20506,qanon,1263904736580952065,Qanon,8.0


In [48]:
hash_consp_mapping = consp_theory_link.loc[:, ['hashtagid', 'theory_id']]

In [49]:
hash_consp_mapping = hash_consp_mapping.drop_duplicates(subset=['hashtagid'])

In [338]:
hash_consp_mapping.dtypes

hashtagid      int64
theory_id    float64
dtype: object

In [339]:
hash_consp_mapping

,hashtagid,theory_id
0,20506,8.0
1,25089,8.0
4,77112,8.0
5,511508,8.0
6,25091,8.0
...,...,...
3633455,22709706,NaN
3638562,22720370,NaN
3640522,22724297,NaN
3648971,22741544,NaN


In [50]:
hash_consp_mapping['hashtagid'] = hash_consp_mapping['hashtagid'].astype(int)
hash_consp_mapping = hash_consp_mapping.dropna()
hash_consp_mapping['theory_id'] = hash_consp_mapping['theory_id'].astype(int)

In [51]:
hash_consp_mapping

,hashtagid,theory_id
1,20506,8
2,25089,8
5,77112,8
6,511508,8
7,25091,8
...,...,...
2518578,20298034,8
2558074,20378669,8
3001741,21370247,8
3483225,22393264,8


In [288]:
for index, row in hash_consp_mapping.iterrows():
    print(row['hashtagid'], row['theory_id'])

20506.0 8.0
25089.0 8.0
77112.0 8.0
511508.0 8.0
25091.0 8.0
14415.0 8.0
15291661.0 nan
21776040.0 nan
259500.0 nan
158957.0 nan
17457016.0 nan
488582.0 nan
6484683.0 nan
11035256.0 nan
15172104.0 nan
3794870.0 nan
15186691.0 nan
33792.0 4.0
27195.0 8.0
18287.0 nan
15198602.0 nan
15195872.0 nan
66445.0 7.0
18270990.0 nan
9723034.0 nan
29024.0 nan
1277842.0 nan
18270450.0 nan
24818.0 8.0
15673159.0 6.0
15236079.0 nan
15487597.0 nan
8744833.0 nan
18589378.0 nan
18387797.0 8.0
15523595.0 nan
15170265.0 nan
15170812.0 nan
21824996.0 nan
227121.0 nan
227122.0 nan
631896.0 nan
18618384.0 nan
18338657.0 nan
259499.0 nan
22761888.0 nan
18270401.0 nan
8756026.0 nan
15185482.0 nan
18378954.0 nan
3073063.0 2.0
18912656.0 nan
15179018.0 nan
22510276.0 nan
15172247.0 nan
16334262.0 8.0
15211530.0 nan
2925634.0 nan
18411406.0 nan
18553018.0 nan
15241876.0 nan
88504.0 8.0
15169536.0 6.0
13410962.0 nan
5310529.0 6.0
17075412.0 nan
5112446.0 2.0
22770416.0 nan
5500249.0 nan
15899791.0 8.0
15560684.0 na

Adding the mapping between theories and hashtags

In [52]:
try:
    engine.execute("""
        BEGIN TRAN

        CREATE TABLE IF NOT EXISTS hash_consptheories_link (
        id int GENERATED ALWAYS AS IDENTITY,
        hashtag_id int,
        consp_theory_id int,
        PRIMARY KEY(id),
        CONSTRAINT fk_hashtag
            FOREIGN KEY(hashtag_id)
                REFERENCES hashtags(id)
                ON DELETE SET NULL,

        CONSTRAINT fk_consptheory        
            FOREIGN KEY(consp_theory_id)
                REFERENCES consp_theories(id)
                ON DELETE SET NULL
    );  

    COMMIT TRAN 
    """)
    
except:
    print("Something went wrong")

In [53]:
try:
    for _, row in hash_consp_mapping.iterrows():
        # print(row['theory_name'])
        engine.execute(f"""
            BEGIN TRAN
            INSERT INTO hash_consptheories_link(hashtag_id, consp_theory_id)
            VALUES ('{int(row['hashtagid'])}', '{int(row['theory_id'])}');
            COMMIT TRAN 
        """)
#         # consp_theory_tab.to_sql('consp_theories', engine, if_exists='replace', index=False)
except:
    print("Somthing went wrong")

Next we will perform the same query but also with joins

After that we will need to parse all the tweet contents and compute the sentiment